In [131]:
!pip3 install torch
!pip3 install pydub
!pip3 install python_speech_features

### Import all packages

In [0]:
# from hw2
import pydub
from pydub import AudioSegment
import pydub
from pydub.playback import play
from python_speech_features import mfcc
from python_speech_features import logfbank
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from time import sleep
import scipy.io.wavfile as wav
from glob import glob
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
%matplotlib inline

In [0]:
import numpy as np
import pandas as pd
import os

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [135]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


### Preparing the dataset

In [136]:
#!ls /content/drive/'My Drive'/validated_clips/validated_clip/1.000000
downloaded_files = glob("/content/drive/My Drive/validated_clips/validated_clip/1.*/*.mp3", recursive=True)
num_downloaded_files = len(downloaded_files)
print(num_downloaded_files)

1000


In [0]:
filepath_dict = {Path(filepath).name : filepath for filepath in downloaded_files}

In [138]:
print(filepath_dict)

{'common_voice_en_10063.mp3': '/content/drive/My Drive/validated_clips/validated_clip/1.000000/common_voice_en_10063.mp3', 'common_voice_en_10062.mp3': '/content/drive/My Drive/validated_clips/validated_clip/1.000000/common_voice_en_10062.mp3', 'common_voice_en_10057.mp3': '/content/drive/My Drive/validated_clips/validated_clip/1.000000/common_voice_en_10057.mp3', 'common_voice_en_10049.mp3': '/content/drive/My Drive/validated_clips/validated_clip/1.000000/common_voice_en_10049.mp3', 'common_voice_en_10083.mp3': '/content/drive/My Drive/validated_clips/validated_clip/1.000000/common_voice_en_10083.mp3', 'common_voice_en_10068.mp3': '/content/drive/My Drive/validated_clips/validated_clip/1.000000/common_voice_en_10068.mp3', 'common_voice_en_10054.mp3': '/content/drive/My Drive/validated_clips/validated_clip/1.000000/common_voice_en_10054.mp3', 'common_voice_en_10050.mp3': '/content/drive/My Drive/validated_clips/validated_clip/1.000000/common_voice_en_10050.mp3', 'common_voice_en_10058.

In [0]:
dev = pd.read_csv("/content/drive/My Drive/tsvs/dev.tsv", sep="\t")
train = pd.read_csv("/content/drive/My Drive/tsvs/train.tsv", sep="\t")
test = pd.read_csv("/content/drive/My Drive/tsvs/test.tsv", sep="\t")
validated = pd.read_csv("/content/drive/My Drive/tsvs/validated.tsv", sep="\t")
invalidated = pd.read_csv("/content/drive/My Drive/tsvs/invalidated.tsv", sep="\t")
#print(dev)
#TODO use numpy arrays instead dataframes
#TODO how are we balancing classes?
splits = {
    "dev": dev,
    "train": train, 
    "test": test,
    "validated": validated,
    "invalidated": invalidated
}

#### Fix the paths

In [140]:
print(splits["dev"]["path"].head())

0    common_voice_en_18540003.mp3
1    common_voice_en_18540005.mp3
2    common_voice_en_18540006.mp3
3       common_voice_en_65557.mp3
4       common_voice_en_65559.mp3
Name: path, dtype: object


In [141]:
#TODO: Fix prefixes
for key in splits:
    splits[key]["path"] = splits[key]["path"].map(filepath_dict)
    splits[key] = splits[key][splits[key]['path'].notna()]
    print(key)
    print(splits[key].shape)

dev
(14, 8)
train
(24, 8)
test
(48, 8)
validated
(1000, 8)
invalidated
(0, 8)


In [0]:
#print(splits)

#### Drop the rows with NaN accent values

In [143]:
for key in splits:
    print(key, len(splits[key]))

dev 14
train 24
test 48
validated 1000
invalidated 0


In [144]:
for key in splits:
    splits[key].dropna(axis=0, subset=["accent"], inplace=True)
    print(key)
    print(len(splits[key]))

dev
2
train
12
test
3
validated
139
invalidated
0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


#### Choose the data we've actually downloaded

In [145]:
for key in splits:
    splits[key] = splits[key][splits[key]["path"].isin(downloaded_files)]
    print(len(splits[key]))

2
12
3
139
0


#### Make sure speaker independent and validated

In [146]:
len(splits['train'][splits['train']["client_id"].isin(splits['test']['client_id'])])

0

In [147]:
len(splits['train'][splits['train']["client_id"].isin(splits['dev']['client_id'])])

0

In [148]:
num_train = len(splits['train'][splits['train']["client_id"].isin(splits['validated']['client_id'])])
num_test = len(splits['test'][splits['test']["client_id"].isin(splits['validated']['client_id'])])
num_dev = len(splits['dev'][splits['dev']["client_id"].isin(splits['validated']['client_id'])])

print(f"Wow, we only end up with {num_train} training, {num_test} test and {num_dev} dev audio files from the {num_downloaded_files} files we started with!")

Wow, we only end up with 12 training, 3 test and 2 dev audio files from the 1000 files we started with!


In [0]:
## For the purposes of this toy example we forgoe using only the validated clips and assume all clips are good. 

# Get MFCC

#### Lets start preprocessing our actual audio files to the dataset

Some useful functions:

In [0]:
from utils import *

##### Extract the audio files, remove leading silence, and silent clips...

In [0]:
# Remove silent clips
useful_keys = ['train','test','dev']
for key in useful_keys:
  drop_idxs = splits[key][splits[key]['path'].apply(detect_leading_silence_filepath) >= splits[key]['path'].apply(length_of_file)-1].index
  splits[key] = splits[key].drop(drop_idxs)

In [0]:
downloaded_audios = {key : [pydub.AudioSegment.from_mp3(f) for f in splits[key]['path']] for key in useful_keys}

In [153]:
print([len(thing) for thing in downloaded_audios.values()])

[12, 3, 2]


In [154]:
max_lengths = [np.max([len(sample) for sample in samples]) for samples in downloaded_audios.values()]
max_audio_length = np.max(max_lengths)
print(max_lengths, max_audio_length)
print(f"max audio length is {max_audio_length / 1000} seconds")

[8136, 5400, 3456] 8136
max audio length is 8.136 seconds


In [0]:
# Zero pad to normalize length
padded_audio_dict = {key: [zero_pad_in_end(audio, max_audio_length) for audio in audios] for key, audios in downloaded_audios.items()}

In [0]:
mfccs_padded = np.array([extract_mfcc(audio) for audio in padded_audios])
mfccs_padded_dict = {key: np.array([extract_mfcc(audio) for audio in padded_audios]) for key, padded_audios in padded_audio_dict.items()}

In [157]:
print([values.shape for values in mfccs_padded_dict.values()])

[(12, 2440, 13), (3, 2440, 13), (2, 2440, 13)]


In [158]:
labels = train['accent'].to_numpy()
labels_dict = {key: splits[key]['accent'].to_numpy() for key in useful_keys}
print([values.shape for values in labels_dict.values()])

[(12,), (3,), (2,)]


In [159]:
paddingamounts = [(len(pad)-len(unpad))/1000 for unpad,pad in zip(downloaded_audios, padded_audios)]
print(paddingamounts)
print(sum(paddingamounts)/len(paddingamounts))

[8.131, 8.132, 8.133]
8.132


In [0]:
import dill

In [0]:
session_path = '/content/drive/My Drive/validated_clips/session.pkl'
dill.dump_session(session_path)

In [162]:
mfccs_padded.shape

(12, 2440, 13)

![alt text](https://)#TODO: Plot some filterbanks and mfccs

#CNN

#### TODO:: Create a validation set?

#### TODO: Creating our Toy Models

##### Toy CNN

In [163]:
mfccs_padded = np.moveaxis(mfccs_padded, (1,2), (2,1))
mfccs_padded.shape
#want array size to be
#numbe of samples, number of filterbanks, length of each sample

(12, 13, 2440)

In [0]:
class downwardSlope(nn.Module):
    def __init__(self, maxSeqLength, outsize):
        super().__init__()
        convOutLength = maxSeqLength
        padding = 1
        dilation = 1
        kernel_size = 15
        stride = 1

        self.conv1 = nn.Conv1d(13, 32, kernel_size=kernel_size, dilation =dilation ,stride=stride, padding=padding)
        #convOutLength calculated using equation in nn.conv1d documentation
        convOutLength = (convOutLength + 2*padding -dilation*(kernel_size-1) -1)//stride + 1
        self.conv2 = nn.Conv1d(32, 32, 15, padding=padding)
        convOutLength = (convOutLength + 2*padding -dilation*(kernel_size-1) -1)//stride + 1
        self.conv3 = nn.Conv1d(32, 32, 15, padding=padding)
        convOutLength = (convOutLength + 2*padding -dilation*(kernel_size-1) -1)//stride + 1

        self.fc1 = nn.Linear(convOutLength, outsize)

    def forward(self,x):
        print("insideforward",type(x), x.dtype)
        x= self.conv1(x)
        x= self.conv2(x)
        x= self.conv3(x)
        x = self.fc1(x)
        x = torch.sigmoid(x)
        return torch.squeeze(x)

In [0]:
def evaluateModel(model, lossCriterion, X, y, batchSize):
    with torch.no_grad():
        n=batchSize
        batched = list(zip([X[i:i + n] for i in range(0, len(X), n)],
                          [y[i:i + n] for i in range(0, len(y), n)]))
        predList = []
        yList = []
        lossMeans = []
        for i,(batchX, batchY) in enumerate(batched):
            print('x',type(X), X.dtype)
            print('batchx',type(batchX), batchX.dtype)
            predProb = model(batchX)
            if np.isnan(predProb.cpu().detach().numpy()).any():
                print("AAAAAA a NAN")
                return
            loss = lossCriterion(predProb, batchY)
            batchPred = predProb.round().tolist()
            predList = predList+batchPred
            batchYargmaxed = batchY.tolist()
            yList = yList+batchYargmaxed
            lossMeans.append(torch.mean(loss).item())

        #acc = accuracy_score(yList, predList)
        #report = classification_report(yList, predList)
        #confMat = confusion_matrix(yList, predList)
        lossMean = sum(lossMeans)/float(len(lossMeans))

    return lossMean

In [0]:
def trainModel(model, optimizer, criterion, train_X, train_y, val_X, val_y, batchSize, startEpoch, endEpoch):
    notifyEvery = 100 if torch.cuda.is_available() else 2

    checkmarkTime = time.time()

    n=batchSize
    batched = list(zip([train_X[i:i + n] for i in range(0, len(train_X), n)],
                      [train_y[i:i + n] for i in range(0, len(train_y), n)]))
    numBatches = len(batched)
    print("number of batches", numBatches)
    for epoch in range(startEpoch, endEpoch):
        print("epoch:", epoch)
        for i,(batchX,batchy) in enumerate(batched):
            optimizer.zero_grad()
            output =  model(batchX)
            loss = criterion(output, batchy)
            loss.backward()
            optimizer.step()
            if np.isnan(output.cpu().detach().numpy()).any():
                print("AAAAAA a NAN")
                return
            if i%notifyEvery ==notifyEvery-1:
                print('[%d, %5d]' %
                  (epoch + 1, i + 1))
                timeTook = time.time() - checkmarkTime
                print("took", timeTook, "seconds for", notifyEvery, "batches")
                if(torch.cuda.is_available()):
                    print(torch.cuda.max_memory_allocated()/1e9, "GB of VRAM being used")
                checkmarkTime = time.time()
        trainLoss = evaluateModel(model,criterion,train_X, train_y, batchSize)
        valLoss = evaluateModel(model, criterion,val_X, val_y, batchSize)
        print('loss', valLoss)
    print("finished training")

In [167]:
!pip3 install skorch

In [0]:
from skorch import NeuralNetClassifier

In [169]:
from sklearn import preprocessing
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import time

X = torch.as_tensor(mfccs_padded, dtype=torch.float)
le = preprocessing.LabelEncoder()
le.fit(labels)
transformed = le.transform(labels)

y = torch.as_tensor(onehotted, dtype=torch.long)
seqLength = mfccs_padded.shape[2]
print(seqLength)
outsize = len(le.classes_)
print(outsize, type(onehotted), type(y), onehotted.shape)
mod = downwardSlope(seqLength, outsize)
print(type(transformed),transformed.shape, print(transformed))
enc =  preprocessing.OneHotEncoder(handle_unknown='ignore')
onehotted =enc.fit_transform(transformed.reshape(-1,1)).toarray() 


device = "cpu"
criterion = nn.CrossEntropyLoss()
endEpoch = 3
learningRate = 0.001
print('x',type(X), X.dtype)
optimizer = torch.optim.Adam(mod.parameters(), lr = learningRate)

trainModel(mod, optimizer,criterion, X[:20], y[:20],X[20:],y[20:], 10, 0, 10)

TypeError: ignored

##### Toy LSTM

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(42)
# https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html